## Início do algoritmo
O primeiro passo é o mais simples. Apenas vou ler os dados utilizando a biblioteca `pandas`.

In [1]:
import pandas as pd

#dados dos filmes e notas
filmes = pd.read_csv('../data/movies.csv')
notas = pd.read_csv('../data/ratings.csv')

## Pré processamento dos dados
Preciso limpar os dados antes de proceguir. Nesse caso, retirar valores vazios utilizando o método `dropna()` da biblioteca `pandas`.

In [3]:
# Etapa de verificação primária

#Valores nulos em notas:
print(notas.isnull().sum())
#Valores nulos em filmes:
print(filmes.isnull().sum())

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64
movieId    0
title      0
genres     0
dtype: int64


Não encontrei **valores ausentes**, logo não vou precisar utilizar o `dropana()` do `pandas`.
Segue um exemplo se caso tivesse **valores vazios** nos dados:

In [4]:
#Remoção de valores nulos
notas = notas.dropna()
filmes = filmes.dropna()